In [1]:
!nvidia-smi

Tue Nov 19 18:10:51 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     On  | 00000000:00:0A.0 Off |                  N/A |
| 27%   31C    P8               2W / 260W |      1MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Script to run code on the cluster

In [2]:
import torch
torch.cuda.is_available()

True

In [17]:
!python compute_prs.py --dataset imagenet --device cuda:0 --model ViT-B-32 --pretrained laion2b_s34b_b79k --data_path datasets/imagenet --cache_dir "../cache/" --samples_per_class 3


^C
Traceback (most recent call last):
  File "/cluster/work/vogtlab/Group/vstrozzi/working-MT2024/compute_prs.py", line 10, in <module>
    from utils.factory import create_model_and_transforms, get_tokenizer
  File "/cluster/work/vogtlab/Group/vstrozzi/working-MT2024/utils/__init__.py", line 2, in <module>
    from utils.factory import create_model, create_model_and_transforms, create_model_from_pretrained, get_tokenizer, create_loss
  File "/cluster/work/vogtlab/Group/vstrozzi/working-MT2024/utils/factory.py", line 9, in <module>
    import timm
  File "/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.12/site-packages/timm/__init__.py", line 2, in <module>
    from .layers import is_scriptable, is_exportable, set_scriptable, set_exportable
  File "/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.12/site-packages/timm/layers/__init__.py", line 8, in <module>
    from .classifier import create_classifier, ClassifierHead, NormMlpClassi

In [12]:
!python compute_text_projection.py  --dataset imagenet --device cuda:0 --model ViT-B-32 --pretrained laion2b_s34b_b79k --cache_dir "../cache/"

/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
Using local files
/cluster/work/vogtlab/Group/vstrozzi/working-MT2024/utils/factory.py:86: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they 

In [15]:
!python compute_text_set_projection.py --device cuda:0 --model ViT-B-32 --pretrained laion2b_s34b_b79k --cache_dir "../cache/" --data_path text_descriptions/laion.txt

/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
Using local files
/cluster/work/vogtlab/Group/vstrozzi/working-MT2024/utils/factory.py:86: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they 

# Test complete text set

In [15]:
!python compute_complete_text_set.py --device cuda:0 --model ViT-B-32 --texts_per_head 100 --num_of_last_layers 4 --text_descriptions image_descriptions_general

In [11]:
import json

# Read JSON lines
with open("output_dir/imagenet_completeness_image_descriptions_general_top_60_heads_ViT-B-32.jsonl", "r") as json_file:
    for line in json_file:
        entry = json.loads(line)  # Parse each line as a JSON object
        layer = entry["layer"]
        head = entry["head"]
        texts = entry["embeddings_sort"]

        print(f"Layer: {layer}, Head: {head}")
        print("Texts:")
        for text in texts:
            print(text)

Layer: 8, Head: 0
Texts:
{'text': 'A cup', 'strength_abs': 0.010872583836317062, 'strength_rel': 3.3802812236019313}
{'text': 'A semicircular arch', 'strength_abs': 0.009496849961578846, 'strength_rel': 2.952566206135948}
{'text': 'Picture snapped in the Australian deserts', 'strength_abs': 0.009106921963393688, 'strength_rel': 2.8313377740847443}
{'text': 'Intricate architectural carving ', 'strength_abs': 0.008942307904362679, 'strength_rel': 2.7801593402128644}
{'text': 'Picture of mammels', 'strength_abs': 0.008655490353703499, 'strength_rel': 2.690987898015815}
{'text': 'An image of the number 4', 'strength_abs': 0.007634803652763367, 'strength_rel': 2.373656880632109}
{'text': 'Rustic charm', 'strength_abs': 0.0076140244491398335, 'strength_rel': 2.367196635955456}
{'text': 'Image with a futuristic time manipulation experiment', 'strength_abs': 0.007117657922208309, 'strength_rel': 2.2128765151570606}
{'text': 'Inviting outdoor seating', 'strength_abs': 0.006945070344954729, 'str

In [7]:
!python compute_complete_text_set.py --device cuda:0 --model ViT-B-32 --texts_per_head 60 --num_of_last_layers 4 --text_descriptions image_descriptions_general

^C
Traceback (most recent call last):
  File "/cluster/work/vogtlab/Group/vstrozzi/working-MT2024/compute_complete_text_set.py", line 15, in <module>
    from torchvision.datasets import ImageNet
  File "/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.12/site-packages/torchvision/__init__.py", line 10, in <module>
    from torchvision import _meta_registrations, datasets, io, models, ops, transforms, utils  # usort:skip
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.12/site-packages/torchvision/models/__init__.py", line 2, in <module>
    from .convnext import *
  File "/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.12/site-packages/torchvision/models/convnext.py", line 8, in <module>
    from ..ops.misc import Conv2dNormActivation, Permute
  File "/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/e

In [42]:
!python compute_complete_text_set.py --device cuda:0 --model ViT-B-32 --texts_per_head 200 --num_of_last_layers 4 --text_descriptions image_descriptions_general

/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
Number of layers: 12
  0%|                                                     | 0/4 [00:00<?, ?it/s][3125, 3125, 3125, 200]
Epoch [20/1500], Loss: 0.233754
Epoch [40/1500], Loss: 0.261123
Epoch [60/1500], Loss: 0.229569
Epoch [80/1500], Loss: 0.223419
Epoch [100/1500], Loss: 0.224193
Epoch [120/1500], Loss: 0.240422
Epoch [140/1500], Loss: 0.229513
Epoch [160/1500], Loss: 0.269355
Epoch [180/1500], Loss: 0.299473
Epoch [200/1500], Loss: 0.275157
Epoch [220/1500], Loss: 0.232199
Epoch [240/1500], Loss: 0.199710
Epoch [260/1500], Loss: 0.189859
Epoch [280/1500], Loss: 0.180053
Epoch [300/1500], Loss: 0.171267
Epoch [320/1500], Loss: 0.176079
Epoch [340/1500], Loss: 